In [142]:
from selenium.webdriver import Chrome, ChromeOptions
from bs4 import BeautifulSoup
import gzip
from pathlib import Path
from datetime import datetime
import re
from glob import glob
import pandas as pd
from time import sleep
from random import uniform

In [188]:
url = 'https://dunksandthrees.com/'
chromedriver_path = Path().joinpath(
    '/Users','johnschroeder','Documents','gambling_stuff','chromedriver')
base_path = '/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_team_ratings/dunks_and_threes'

In [106]:
chrome_options = ChromeOptions()
# chrome_options.add_argument('--headless')

In [107]:
driver = Chrome(executable_path=chromedriver_path, options=chrome_options)

In [130]:
response = driver.get(url)

In [148]:
soup = BeautifulSoup(driver.page_source)

In [149]:
raw_page_date = soup.find('span', {'class': 'page-date'}).text

In [137]:
ymd = str(datetime.strptime(raw_page_date, '%B %d, %Y').date()).split('-')

In [154]:
raw_file = Path(base_path).joinpath(*ymd).with_suffix('.html')

In [114]:
Path(raw_file).parent.mkdir(parents=True, exist_ok=True)
with open(raw_file, 'w') as fp:
    fp.write(str(team_stats.prettify()))

In [113]:
team_stats = soup.find('div', {'id': 'team-stats'})

In [147]:
soup = BeautifulSoup(driver.page_source)
raw_page_date = soup.find('span', {'class': 'page-date'}).text
print(raw_page_date)
ymd = str(datetime.strptime(raw_page_date, '%B %d, %Y').date()).split('-')
raw_file = Path(base_path).joinpath(*ymd, 'dunksandthrees.html')
team_stats = soup.find('div', {'id': 'team-stats'})
Path(raw_file).parent.mkdir(parents=True, exist_ok=True)
with open(raw_file, 'w') as fp:
    fp.write(str(team_stats.prettify()))

July 30, 2020


In [198]:
# iterate through multiple clicks to pull multiple raw files
nbr_clicks = 5

for click in range(nbr_clicks):
    soup = BeautifulSoup(driver.page_source)
    raw_page_date = soup.find('span', {'class': 'page-date'}).text
    print(raw_page_date)
    ymd = str(datetime.strptime(raw_page_date, '%B %d, %Y').date()).split('-')
    raw_file = Path(base_path).joinpath(*ymd).with_suffix('.html')
    team_stats = soup.find('div', {'id': 'team-stats'})
    Path(raw_file).parent.mkdir(parents=True, exist_ok=True)
    with open(raw_file, 'w') as fp:
        fp.write(str(team_stats.prettify()))
    driver.find_element_by_class_name('date-button').click()
    sleep(uniform(1.5, 2.5))

October 21, 2017
October 20, 2017
October 19, 2017
October 18, 2017
October 17, 2017


In [115]:
# collect all raw files and combine to a single dataframe

In [116]:
curated_file = '/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv'

In [117]:
stats_headers = team_stats.find('tr', {'class': 'main-header-tr'})

In [118]:
df_headers = [re.sub('\W+', '', _.text).lower() for _ in stats_headers.find_all('th')]

In [119]:
df_headers

['team',
 'wl',
 'anet',
 'aortg',
 'adrtg',
 'sos',
 'osos',
 'dsos',
 'pace',
 'opl',
 'dpl']

In [219]:
df_headers1 = ['team',
 'wl',
 'anet',
 'aortg',
 'adrtg',
 'sos',
 'osos',
 'dsos',
 'pace',
 'opl',
 'dpl',
 'date']

In [204]:
'-'.join(Path('/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_team_ratings/dunks_and_threes/2021/05/10.html').with_suffix('').parts[-3:])

'2021-05-10'

In [217]:
raw_files = glob('/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_team_ratings/dunks_and_threes/*/*/*.html')
df_data = []
for raw_file in raw_files:
    with open(raw_file, 'r') as fp:
        raw_html = fp.read()
    team_stats = BeautifulSoup(raw_html)
    team_rows = team_stats.find_all('tr', {'class': 'stat-row'})
    replaces = re.compile(r'(\+)|(\s+)')
    file_date = '-'.join(Path(raw_file).with_suffix('').parts[-3:])
    for team_row in team_rows:
        row_data = [re.sub(replaces, '', _.text).strip() for _ in team_row.find_all('td')]
        row_data.append(file_date)
        df_data.append(row_data)

In [121]:
team_rows = team_stats.find_all('tr', {'class': 'stat-row'})

In [122]:
team_rows[0]

<tr class="stat-row entered" style="opacity: 1;"><td class="team-td" style="padding: 0px 0.2em;"><span style="vertical-align: middle;"><span class="small-span">1</span> UTA </span><img alt="" src="images/teams/UTA.png" style="vertical-align: middle;" width="20px"/></td><td class="wl-td">50-19</td><td class="stat-td adj_netrtg"><span class="val-span">+8.9</span><span class="rank-span" style="background-color: rgb(28, 118, 60);">1</span></td><td class="stat-td adj_ortg"><span class="val-span">117.1</span><span class="rank-span" style="background-color: rgb(29, 87, 51);">5</span></td><td class="stat-td adj_drtg right-border"><span class="val-span">108.2</span><span class="rank-span" style="background-color: rgb(28, 107, 57);">3</span></td><td class="stat-td sos"><span class="val-span">-0.0</span><span class="rank-span" style="background-color: rgb(29, 30, 34);">13</span></td><td class="stat-td sos_o"><span class="val-span">-0.0</span><span class="rank-span" style="background-color: rgb(29

In [123]:
replaces = re.compile(r'(\+)|(\d+\s+)')
df_data = []
for team_row in team_rows:
    row_data = [re.sub(replaces, '', _.text).strip() for _ in team_row.find_all('td')]
    df_data.append(row_data)

In [220]:
incr_df = pd.DataFrame(df_data, columns=df_headers1)

In [224]:
incr_df['team'].unique()

array(['HOU', 'GSW', 'OKC', 'BOS', 'TOR', 'POR', 'WAS', 'NYK', 'IND',
       'DET', 'SAS', 'UTA', 'DEN', 'PHI', 'ORL', 'MIN', 'CHA', 'NOP',
       'LAC', 'CLE', 'MEM', 'LAL', 'MIL', 'BKN', 'MIA', 'ATL', 'DAL',
       'PHX', 'CHI', 'SAC'], dtype=object)

In [222]:
incr_df['team'] = incr_df['team'].replace(regex='\d+', value='')

In [124]:
incr_df = pd.DataFrame(df_data, columns=df_headers)
incr_df['date'] = str(datetime.strptime(raw_page_date, '%B %d, %Y').date())

In [125]:
current_curated_df = pd.read_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv')

In [126]:
current_curated_df.head()

,team,wl,anet,aortg,adrtg,sos,osos,dsos,pace,opl,dpl,date
0,OKC,21-48,-10.330,103.230,113.522,0.510,0.111,0.410,101.370,14.312,14.120,2021-05-09
1,GSW,35-33,0.515,110.223,109.750,-0.116,-0.125,0.080,102.520,13.630,14.517,2021-05-09
2,LAC,45-23,6.120,117.140,111.090,-0.114,-0.020,-0.017,97.427,15.026,14.618,2021-05-09
3,PHI,47-21,5.630,113.113,107.520,-0.012,-0.019,-0.016,99.812,14.716,14.110,2021-05-09
4,PHX,48-20,5.640,116.570,111.080,-0.122,0.180,-0.227,97.426,14.822,14.727,2021-05-09


In [127]:
incr_df.head()

,team,wl,anet,aortg,adrtg,sos,osos,dsos,pace,opl,dpl,date
0,UTA,50-19,8.91,117.15,108.23,-0.013,-0.018,-0.014,99.115,14.719,14.37,2021-05-10
1,LAC,45-23,6.12,117.14,111.09,-0.115,-0.021,-0.016,97.427,15.026,14.618,2021-05-10
2,PHI,47-21,5.63,113.113,107.52,-0.014,-0.020,-0.015,99.812,14.716,14.11,2021-05-10
3,PHX,48-20,5.64,116.67,111.08,-0.122,0.18,-0.327,97.426,14.822,14.727,2021-05-10
4,MIL,43-25,5.15,116.86,111.710,-0.329,-0.022,-0.328,102.33,13.62,14.619,2021-05-10


In [128]:
revised_df = pd.concat([current_curated_df, incr_df]).sort_values('date', ascending=False)

In [225]:
incr_df.sort_values('date', ascending=False).to_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv', index=False)

In [129]:
revised_df.to_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv', index=False)